## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df.groupby('APPLICATION_TYPE')['ASK_AMT'].nunique().sort_values(ascending=False)

APPLICATION_TYPE
T3     7423
T6      370
T5      298
T4      234
T9      142
T7      110
T8       78
T19      72
T10      22
T12      15
T13      13
T2       11
T14       3
T25       3
T29       2
T15       2
T17       1
Name: ASK_AMT, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = ['T9', 'T7', 'T8', 'T19', 'T10', 'T12', 'T13', 'T2', 'T14', 'T25', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     3331
T4        1542
T6        1216
T5        1173
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df.groupby('CLASSIFICATION')['ASK_AMT'].nunique().sort_values(ascending=False)

CLASSIFICATION
C1000    5074
C2000    1490
C1200     927
C3000     414
C2100     306
         ... 
C1820       1
C2150       1
C2170       1
C2190       1
C8210       1
Name: ASK_AMT, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df.groupby('CLASSIFICATION')['ASK_AMT'].nunique().sort_values(ascending=False).head(36)

CLASSIFICATION
C1000    5074
C2000    1490
C1200     927
C3000     414
C2100     306
C7000     263
C4000      77
C1700      71
C2800      35
C2700      32
C1270      29
C1300      19
C7100      15
C7200      13
C1800      13
C1500      11
C1400      11
C5000      11
C7120       8
C8000       8
C2300       5
C1280       5
C1240       4
C2400       3
C2710       3
C8200       3
C4100       3
C7210       3
C3200       2
C0          2
C1600       2
C6000       2
C1250       2
C1238       2
C1234       2
C1230       2
Name: ASK_AMT, dtype: int64

In [7]:
class_data = application_df.groupby('CLASSIFICATION')['ASK_AMT'].nunique().sort_values(ascending=False)
class_df = pd.DataFrame(class_data)
# class_df['ASK_AMT'] = class_df['ASK_AMT'].astype(int)
class_df=class_df.reset_index()
class_df

,CLASSIFICATION,ASK_AMT
0,C1000,5074
1,C2000,1490
2,C1200,927
3,C3000,414
4,C2100,306
...,...,...
66,C1820,1
67,C2150,1
68,C2170,1
69,C2190,1


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
for row in class_df.index:
    if class_df['ASK_AMT'][row] < 263:
      classifications_to_replace.append(class_df['CLASSIFICATION'][row])
    else:
      continue

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)
application_df_dummies.head(10)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
5,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,1,31452,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
7,1,7508025,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
8,1,94389,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
9,1,5000,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df_dummies['IS_SUCCESSFUL']
x = application_df_dummies.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=40))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3280      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,741
Trainable params: 5,741
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5806 - accuracy: 0.7131
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5650 - accuracy: 0.7226
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5618 - accuracy: 0.7245
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5601 - accuracy: 0.7246
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7235
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5573 - accuracy: 0.7264
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5575 - accuracy: 0.7264
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5570 - accuracy: 0.7266
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5556 - accuracy: 0.7271
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5562 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5714 - accuracy: 0.7258 - 346ms/epoch - 1ms/step
Loss: 0.5714001655578613, Accuracy: 0.7258309125900269


In [21]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")
